# merge segmentations UKB

In [1]:
import os

import nibabel  # for loading and saving nifty files
import torch
from math import ceil, floor
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import Pad
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import nrrd

In [2]:
print(os.getcwd())

/home/anne/phd/projects/whole_body/UKB_scans


## combination of segmentation files:
creates new segmentation image of the same size as the merged volume  
changes the labels of the segmentation maps according to label list below  
also reads header file of .nrrd files.  
merges segmentations according to header['space origin']

### labels:
liver = 1  
spleen = 2  
kidney_r = 3  
kidney_l = 4  
adrenal_r = 5  
adrenal_l = 6  
pancreas = 7  
gallbladder = 8

In [6]:
def combine_segmentations(seg_files, folder, data_dir):
    
    comb_file = [f for f in seg_files if 'comb_01.nii.gz' in f]
    print('comb file: ', comb_file)
    if len(comb_file) == 1:
        comb_file = comb_file[0]
    else:
        print('less or more than 1 comb file found')
        return
    comb_img = nibabel.load(os.path.join(data_dir, folder, comb_file))
    #print(img)
    comb_img_h = comb_img.header
    print('x: ', comb_img_h['qoffset_x'])
    print('y: ', comb_img_h['qoffset_y'])
    print('z: ', comb_img_h['qoffset_z'])

    
    segmentations = []
    for file in seg_files:
        if "nrrd" in file:
            seg, header = nrrd.read(os.path.join(data_dir, folder, file))
            
            # change labels according to label list above. liver stays the same, as it gets label=1         
            seg = change_label(seg, file)
            
            # print out some important header information:
            print(file)
            space_directions = header['space directions']
            space_origin = header['space origin']
            sizes = header['sizes']
            print('z: ', space_origin[2])
            print('size z: ', sizes[2])
            print('spacing z: ', space_directions[2,2])

            segmentations.append((seg,header)) # append tuple of seg and header, need header later
    #print(segmentations)
    
    #print(len(segmentations))
    num_segm = len(segmentations)
    if num_segm < 8:
        print('less than 8 segmentations')
             

    segm_sorted = sorted(segmentations, key=lambda seg: seg[1]['space origin'][2], reverse=True)
    
    comb_segm = np.zeros_like(comb_img.get_data())
    print(comb_segm.shape)
    
    comb_offz = comb_img_h['qoffset_z']
    comb_size = comb_img.shape[2]
    
    
    for seg in segm_sorted:
        segm = seg[0]
        header = seg[1]
        segm_offz = header['space origin'][2]
        segm_size = header['sizes'][2]
        
        if segm_size == comb_size:
            comb_segm += segm
        else:
            diff = comb_size - segm_size
            
            if abs(segm_offz) < abs(comb_offz):
                comb_segm[:,:, diff:] += segm
            else:
                comb_segm[:,:, 0:segm_size] += segm
                
    empty_header = nibabel.Nifti1Header()
    new_img_nii = nibabel.Nifti1Image(comb_segm, comb_img.affine, empty_header)
    new_img_nii.header['pixdim'] = comb_img_h['pixdim']

    print(new_img_nii.header)
    new_file_path = os.path.join(data_dir,folder,'combined_segmentation.nii.gz')

    nibabel.save(new_img_nii, new_file_path)

Caution: does not consider spelling mistakes ;)

In [12]:
def change_label(seg, f):
    if 'spleen' in f or 'Spleen' in f:
        seg[seg == 1] = 2
    
    elif ('kidney' in f or 'Kidney' in f) and ('right' in f  or 'Right' in f):
        seg[seg==1] = 3
        
    elif ('kidney' in f or 'Kidney' in f) and ('left' in f  or 'Left' in f): 
        seg[seg==1] = 4
    
    elif ('adrenal' in f or 'Adrenal' in f) and ('right' in f  or 'Right' in f): 
        seg[seg==1] = 5
        
    elif ('adrenal' in f or 'Adrenal' in f) and ('left' in f  or 'Left' in f):
        seg[seg==1] = 6
        
    elif ('pancreas' in f or 'Pancreas' in f):
        seg[seg==1] = 7
    
    elif ('gallbladder' in f or 'Gallbladder' in f):
        seg[seg==1] = 8
        
    return seg

seg_dir = directory of the segmentation files, also should include the merged volume ...comb_01.nii.gz

In [13]:
seg_dir = '/home/anne/phd/projects/whole_body/UKB_scans/100017/'

listFiles = os.listdir(seg_dir)
sequence = 'opp'
seg_files = [f for f in listFiles if ".nrrd" in f or '.nii.gz' in f]
print(listFiles)

    
combine_segmentations(seg_files, folder, seg_dir)

    



['1004466_20201_2_0', '1013391_20201_2_0', '1005379_20201_2_0', '1003082_20201_2_0', '1942395_20201_2_0', '1004985_20201_2_0', '1002359_20201_2_0', '1013250_20201_2_0', '1883132_20201_2_0', '1584559_20201_2_0']
comb file:  ['Dixon_BH_17s_in_Dixon_BH_17_comb_01.nii.gz']
x:  -247.76788
y:  -191.9643
z:  -462.75
1004466_Dixon_BH_17s_in_Dixon_BH_17_comb_01_Kidney (left).nrrd
z:  -462.75
size z:  81
spacing z:  4.5
1004466_20201_2_0Dixon_BH_17s_opp_Dixon_BH_17s. nii.gz Gallbladder.nrrd
z:  -462.75
size z:  81
spacing z:  4.5
1004466_Dixon_BH_17s_F_Dixon_BH_17sa.nii.nrrd_Adrenal gland (left).nrrd
z:  -294.75
size z:  44
spacing z:  4.5
1004466_Dixon_BH_17s_F_Dixon_BH_17sa.nii.nrrd_Adrenal gland (right).nrrd
z:  -294.75
size z:  44
spacing z:  4.5
1004466_Dixon_BH_17s_F_Dixon_BH_17sa.nii.nrrd_Pancreas.nrrd
z:  -294.75
size z:  44
spacing z:  4.5
1004466_20201_2_0Dixon_BH_17s_opp_Dixon_BH_17s. nii.gz Spleen1_ignacio-2.nrrd
z:  -294.75
size z:  44
spacing z:  4.5
1004466_Dixon_BH_17s_in_Dixon_B

(224, 174, 81)
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3 224 174  81   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [1.       2.232143 2.232143 4.5      0.00669  0.       0.       0.      ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x       : -247.76788
qoffset_y       : -191.9643
q

(224, 174, 81)
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3 224 174  81   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [1.       2.232143 2.232143 4.5      0.00669  0.       0.       0.      ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x       : -247.76788
qoffset_y       : -191.9643
q

In [80]:
seg_dir = '/home/anne/phd/projects/whole_body/UKB_scans/100017/'

listFiles = os.listdir(seg_dir)
sequence = 'opp'
seg_files = [f for f in listFiles if ".nrrd" in f or '.nii.gz' in f]
#print(listFiles)

data_dir = seg_dir
img_file = [f for f in seg_files if '.nii.gz' in f]
print('img file: ', img_file)
if len(img_file) == 1:
    img_file = img_file[0]
else:
    print('less or more than 1 img file found')
    

img = nibabel.load(os.path.join(data_dir, img_file))
#print(img)
img_h = img.header
#print(img_h)
print('x: ', img_h['qoffset_x'])
print('y: ', img_h['qoffset_y'])
print('z: ', img_h['qoffset_z'])

# somehow this is wrong. x, and y should be positive and z negative

segmentations = []
for file in seg_files:
    if "nrrd" in file:
        seg, header = nrrd.read(os.path.join(data_dir, file))

        # change labels according to label list above. liver stays the same, as it gets label=1         
        seg = change_label(seg, file)

        # print out some important header information:
        ##print(file)
        space_directions = header['space directions']
        space_origin = header['space origin']
        sizes = header['sizes']
        #print('z: ', space_origin[2])
        #print('size z: ', sizes[2])
        #print('spacing z: ', space_directions[2,2])

        segmentations.append((seg,header)) # append tuple of seg and header, need header later
#print(segmentations)

#print(len(segmentations))
num_segm = len(segmentations)
if num_segm < 8:
    print('less than 8 segmentations')
    
segm_sorted = sorted(segmentations, key=lambda seg: seg[1]['space origin'][2], reverse=True)

comb_segm = np.zeros_like(img.get_data())
print('final shape: ', comb_segm.shape)

img_dim_v = img.shape


for seg in segm_sorted:
    segm = seg[0]
    header = seg[1]
    segm_off = header['space origin']
    segm_dim_v = header['sizes']
    segm_spacing = abs(np.diag(header['space directions']))

    # if same size, just add the segmentation
    if all(segm_dim_v == img_dim_v):
        print('same')
        comb_segm += segm
    else:
        print('im dim v: ', img_dim_v)
        print('segm dim v: ', segm_dim_v)
        
        im_spacing = abs(img_h['pixdim'][1:4])
        print('im spacing: ', im_spacing)
        print('segm spacing: ', segm_spacing)
        
        im_dim_w = img_dim_v * im_spacing
        segm_dim_w = segm_dim_v * segm_spacing
        
        print('im dim w: ', im_dim_w)
        print('segm dim w: ', segm_dim_w)
        
        # correction of the wrong information in header file
        im_offx = abs(img_h['qoffset_x'])
        im_offy = abs(img_h['qoffset_y'])
        im_offz = img_h['qoffset_z']

        
        im_offset = np.array([im_offx, im_offy, im_offz])
        im_start = im_offset
        im_end = np.array([im_start[0]-im_dim_w[0], im_start[1]-im_dim_w[1], im_start[2]+im_dim_w[2]])
        
        segm_start = segm_off
        segm_end = np.array([segm_start[0]-segm_dim_w[0], segm_start[1]-segm_dim_w[1], segm_start[2]+segm_dim_w[2]])
        
        print('im off: ', im_offset)
        print('segm off: ', segm_off)
        
        print('im start: ', im_start)
        print('im end: ', im_end)
        print('segm start: ', segm_start)
        print('segm end: ', segm_end)
        
        start_diff_w = abs(im_start - segm_start)
        end_diff_w = abs(im_end - segm_end)
        print('start diff w: ', start_diff_w)
        
        print('end_diff w ', end_diff_w)
        
        start_diff_v = np.round(start_diff_w / segm_spacing).astype(int)
        print('start diff v: ', start_diff_v)
        
        end_diff_v = np.round(end_diff_w / segm_spacing).astype(int)
        print('end diff v: ', end_diff_v)
        
        segm_end_v = img_dim_v - end_diff_v
        print('segm end v: ', segm_end_v)
        
        segm_end_x = segm_end_v[0]
        segm_end_y = segm_end_v[1]
        segm_end_z = segm_end_v[2]
        segm_start_x = segm_end_x - segm_dim_v[0]
        segm_start_y = segm_end_y - segm_dim_v[1]
        segm_start_z = segm_end_z - segm_dim_v[2]
        print('segm start v: ', segm_start_x, segm_start_y, segm_start_z)
        
        comb_segm[segm_start_x:segm_end_x, segm_start_y:segm_end_y, segm_start_z:segm_end_z] += segm

empty_header = nibabel.Nifti1Header()
new_img_nii = nibabel.Nifti1Image(comb_segm, img.affine, empty_header)
new_img_nii.header['pixdim'] = img_h['pixdim']

#print(new_img_nii.header)
new_file_path = os.path.join(data_dir,'combined_segmentation.nii.gz')
nibabel.save(new_img_nii, new_file_path)



img file:  ['3D_GRE_TRA_opp_3D_GRE_TRA_2.nii.gz']
x:  -216.09375
y:  -178.90625
z:  -664.5
final shape:  (320, 260, 96)
im dim v:  (320, 260, 96)
segm dim v:  [55 49 28]
im spacing:  [1.40625 1.40625 3.     ]
segm spacing:  [1.40625 1.40625 3.     ]
im dim w:  [450.    365.625 288.   ]
segm dim w:  [77.34375 68.90625 84.     ]
im off:  [ 216.09375  178.90625 -664.5    ]
segm off:  [ 119.41934967  106.36636353 -503.68216324]
im start:  [ 216.09375  178.90625 -664.5    ]
im end:  [-233.90625 -186.71875 -376.5    ]
segm start:  [ 119.41934967  106.36636353 -503.68216324]
segm end:  [  42.07559967   37.46011353 -419.68216324]
start diff w:  [ 96.67440033  72.53988647 160.81783676]
end_diff w  [275.98184967 224.17886353  43.18216324]
start diff v:  [69 52 54]
end diff v:  [196 159  14]
segm end v:  [124 101  82]
segm start v:  69 52 54
im dim v:  (320, 260, 96)
segm dim v:  [55 49 28]
im spacing:  [1.40625 1.40625 3.     ]
segm spacing:  [1.40625 1.40625 3.     ]
im dim w:  [450.    365.625